In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.notebook import tqdm
from BPE_PPGI.utils.jsondataset import JSONDataset
from BIGRAM.bigram_base import BigramBase
from transformers import AutoTokenizer
import os
import random
import shutil
import nltk
import numpy as np


In [ ]:
# Cálculo de perplexidade explícito. O restante foi colocado em arquivos do projeto
def perplexity(model, dataset):
    for text in dataset:
        phrases = nltk.tokenize.sent_tokenize(text, language="portuguese")
        probs_frases = []
        probs_frases_inf = []
        for phrase in phrases:
            probs = 1.0
            count = 0
            tokens = model.tokenizer.encode(phrase)
            for i in range(1, len(tokens)):
                probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]
                count += 1
            aux_prob = np.power(probs, (1/count))
            if np.isinf(aux_prob):
                probs_frases_inf.append(phrase)
            else:
                probs_frases.append(aux_prob)
    return probs_frases, probs_frases_inf

In [2]:
corpus_dir = os.path.join(os.path.abspath('.'), '..', '..', 'dados', 'corpus')
train_dir = os.path.join(os.path.abspath('.'), '..', '..', 'dados', 'train')
evaluation_dir = os.path.join(os.path.abspath('.'), '..', '..', 'dados', 'evaluation')
test_dir = os.path.join(os.path.abspath('.'), '..', '..', 'dados', 'test')

tokenizer_base_dir = os.path.join(os.path.abspath('.'), '..', '..', 'tokenizers')


In [3]:
train_ds = JSONDataset(train_dir)
evaluation_ds = JSONDataset(evaluation_dir)
test_ds = JSONDataset(test_dir)

In [4]:
base_tokenizer = AutoTokenizer.from_pretrained("roberta-base")


In [34]:
tokenizer_300 = base_tokenizer.train_new_from_iterator(train_ds, 300)
tokenizer_300.save_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_300'))

('/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/tokenizer_config.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/special_tokens_map.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/vocab.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/merges.txt',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/added_tokens.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_300/tokenizer.json')

In [35]:
bg_300 = BigramBase(tokenizer=tokenizer_300)

In [36]:
bg_300.train(train_ds)

100%|██████████| 5957/5957 [01:04<00:00, 92.34it/s] 


In [37]:
p, c = perplexity(bg_300, evaluation_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)

/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  26.85913944289181
Std:  17.573890970622486
Max:  261.25699462488825
Min:  12.363841412238024
Total INF 5
Infs:  ['{{Info/Personagem animangá | nome = Son Goku | nome japonês = 孫悟空 | série = Dragon Ball | imagem = SonGoku.jpg | tamanho = 250px | legenda = Goku em sua infância e em sua fase adulta | nome-pt = | primeira = Dragon Ball: capítulo 1 Dragon Ball: episódio 1 | última = Dragon Ball Super: capítulo 76 Dragon Ball Super: episódio 131 | criador = Akira Toriyama | vozes = | outrosnomes = Kakarotto | sexo = Masculino | espécie = Saiyajin | local-nascimento = Planeta Vegeta | ocupação = Lutador de artes marciais Agricultor | família = Bardock Gine Raditz Son Gohan Chi-Chi Son Gohan Son Goten Videl Pan Rei Cutelo Goku Jr. | membro-de = Guerreiros Z|cor de fundo da célula=#FF4500|cor de fundo do título=#FF4500|cor do texto do título=#000|cor do texto da célula=#000}} , cujo nome de nascimento é , é o protagonista da franquia Dragon Ball, criada por Akira Toriyama.', 'A inspiraçã

In [23]:
tokenizer_500 = base_tokenizer.train_new_from_iterator(train_ds, 500)
tokenizer_500.save_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_500'))

('/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/tokenizer_config.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/special_tokens_map.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/vocab.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/merges.txt',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/added_tokens.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_500/tokenizer.json')

In [5]:
tokenizer_500 = AutoTokenizer.from_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_500'))

In [ ]:
bg_500 = BigramBase(tokenizer=tokenizer_500)


In [ ]:
bg_500.train(train_ds)

100%|██████████| 5957/5957 [01:16<00:00, 77.57it/s] 


In [ ]:
p, c = perplexity(bg_500, evaluation_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)


/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  52.57370865281882
Std:  19.53882323699025
Max:  228.72862657470495
Min:  14.349397728027562
Total INF 4
Infs:  ['{{Info/Personagem animangá | nome = Son Goku | nome japonês = 孫悟空 | série = Dragon Ball | imagem = SonGoku.jpg | tamanho = 250px | legenda = Goku em sua infância e em sua fase adulta | nome-pt = | primeira = Dragon Ball: capítulo 1 Dragon Ball: episódio 1 | última = Dragon Ball Super: capítulo 76 Dragon Ball Super: episódio 131 | criador = Akira Toriyama | vozes = | outrosnomes = Kakarotto | sexo = Masculino | espécie = Saiyajin | local-nascimento = Planeta Vegeta | ocupação = Lutador de artes marciais Agricultor | família = Bardock Gine Raditz Son Gohan Chi-Chi Son Gohan Son Goten Videl Pan Rei Cutelo Goku Jr. | membro-de = Guerreiros Z|cor de fundo da célula=#FF4500|cor de fundo do título=#FF4500|cor do texto do título=#000|cor do texto da célula=#000}} , cujo nome de nascimento é , é o protagonista da franquia Dragon Ball, criada por Akira Toriyama.', 'A inspiração

In [22]:
tokenizer_1000 = base_tokenizer.train_new_from_iterator(train_ds, 1000)
tokenizer_1000.save_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_1000'))

('/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/tokenizer_config.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/special_tokens_map.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/vocab.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/merges.txt',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/added_tokens.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_1000/tokenizer.json')

In [23]:
bg1000 = BigramBase(tokenizer=tokenizer_1000)

In [24]:
bg1000.train(train_ds)

100%|██████████| 5957/5957 [00:52<00:00, 113.56it/s]


In [25]:
p, c = perplexity(bg1000, evaluation_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)

/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  75.15271798585407
Std:  31.937027090069783
Max:  309.41055400942327
Min:  21.421964513167623
Total INF 4
Infs:  ['{{Info/Personagem animangá | nome = Son Goku | nome japonês = 孫悟空 | série = Dragon Ball | imagem = SonGoku.jpg | tamanho = 250px | legenda = Goku em sua infância e em sua fase adulta | nome-pt = | primeira = Dragon Ball: capítulo 1 Dragon Ball: episódio 1 | última = Dragon Ball Super: capítulo 76 Dragon Ball Super: episódio 131 | criador = Akira Toriyama | vozes = | outrosnomes = Kakarotto | sexo = Masculino | espécie = Saiyajin | local-nascimento = Planeta Vegeta | ocupação = Lutador de artes marciais Agricultor | família = Bardock Gine Raditz Son Gohan Chi-Chi Son Gohan Son Goten Videl Pan Rei Cutelo Goku Jr. | membro-de = Guerreiros Z|cor de fundo da célula=#FF4500|cor de fundo do título=#FF4500|cor do texto do título=#000|cor do texto da célula=#000}} , cujo nome de nascimento é , é o protagonista da franquia Dragon Ball, criada por Akira Toriyama.', 'A inspiraçã

In [26]:
tokenizer_5000 = base_tokenizer.train_new_from_iterator(train_ds, 5000)
tokenizer_5000.save_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_5000'))

('/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/tokenizer_config.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/special_tokens_map.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/vocab.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/merges.txt',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/added_tokens.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_5000/tokenizer.json')

In [27]:
bg5000 = BigramBase(tokenizer=tokenizer_5000)

In [28]:
bg5000.train(train_ds)

  0%|          | 0/5957 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1005 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 5957/5957 [00:48<00:00, 123.17it/s]


In [29]:
p, c = perplexity(bg5000, evaluation_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)

/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  249.93045395911435
Std:  115.0683796662662
Max:  942.4493757442559
Min:  44.920674797954305
Total INF 4
Infs:  ['{{Info/Personagem animangá | nome = Son Goku | nome japonês = 孫悟空 | série = Dragon Ball | imagem = SonGoku.jpg | tamanho = 250px | legenda = Goku em sua infância e em sua fase adulta | nome-pt = | primeira = Dragon Ball: capítulo 1 Dragon Ball: episódio 1 | última = Dragon Ball Super: capítulo 76 Dragon Ball Super: episódio 131 | criador = Akira Toriyama | vozes = | outrosnomes = Kakarotto | sexo = Masculino | espécie = Saiyajin | local-nascimento = Planeta Vegeta | ocupação = Lutador de artes marciais Agricultor | família = Bardock Gine Raditz Son Gohan Chi-Chi Son Gohan Son Goten Videl Pan Rei Cutelo Goku Jr. | membro-de = Guerreiros Z|cor de fundo da célula=#FF4500|cor de fundo do título=#FF4500|cor do texto do título=#000|cor do texto da célula=#000}} , cujo nome de nascimento é , é o protagonista da franquia Dragon Ball, criada por Akira Toriyama.', 'A inspiração

In [30]:
tokenizer_20_000 = base_tokenizer.train_new_from_iterator(train_ds, 20_000)
tokenizer_20_000.save_pretrained(os.path.join(tokenizer_base_dir, 'tokenizer_roberta_base_20_000'))

('/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/tokenizer_config.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/special_tokens_map.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/vocab.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/merges.txt',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/added_tokens.json',
 '/home/gvanerven/code/PPGI0119-Topicos/Atividade_2/notebooks/../../tokenizers/tokenizer_roberta_base_20_000/tokenizer.json')

In [31]:
bg20_000 = BigramBase(tokenizer=tokenizer_20_000)

In [32]:
bg20_000.train(train_ds)

  0%|          | 0/5957 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (663 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 5957/5957 [00:49<00:00, 119.86it/s]


In [33]:
p, c = perplexity(bg20_000, evaluation_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)

/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  1238.8296391974575
Std:  680.4011577803719
Max:  4172.223368872158
Min:  124.42976225928719
Total INF 4
Infs:  ['{{Info/Personagem animangá | nome = Son Goku | nome japonês = 孫悟空 | série = Dragon Ball | imagem = SonGoku.jpg | tamanho = 250px | legenda = Goku em sua infância e em sua fase adulta | nome-pt = | primeira = Dragon Ball: capítulo 1 Dragon Ball: episódio 1 | última = Dragon Ball Super: capítulo 76 Dragon Ball Super: episódio 131 | criador = Akira Toriyama | vozes = | outrosnomes = Kakarotto | sexo = Masculino | espécie = Saiyajin | local-nascimento = Planeta Vegeta | ocupação = Lutador de artes marciais Agricultor | família = Bardock Gine Raditz Son Gohan Chi-Chi Son Gohan Son Goten Videl Pan Rei Cutelo Goku Jr. | membro-de = Guerreiros Z|cor de fundo da célula=#FF4500|cor de fundo do título=#FF4500|cor do texto do título=#000|cor do texto da célula=#000}} , cujo nome de nascimento é , é o protagonista da franquia Dragon Ball, criada por Akira Toriyama.', 'A inspiração

### Base de Teste

In [38]:
# Aplicando no teste o bg_300
p, c = perplexity(bg_300, test_ds)
print("Mean: ", np.mean(p))
print("Std: ", np.std(p))
print("Max: ", np.max(p))
print("Min: ", np.min(p))
print("Total INF", len(c))
print("Infs: ", c)

/tmp/ipykernel_40018/259248560.py:11: RuntimeWarning: overflow encountered in scalar divide
  probs = probs*1/model.sp_matrix_bigram_prob[tokens[i-1], tokens[i]]


Mean:  19.554183843039578
Std:  2.6084397052454245
Max:  26.767784915886914
Min:  15.780889171593806
Total INF 2
Infs:  ['{{Info/Município do Brasil | nome = Lindóia | nome_oficial = Município da Estância Hidromineral de Lindóia | preposição = de | foto = | leg_foto = Igreja Matriz de Nossa Senhora das Brotas | apelido = Capital Nacional da Água MineralPrefeitura de Lindóia | brasão = Brasao Lindoia SP.PNG | bandeira = Bandeira de Lindóia.jpg | link_brasão = | link_bandeira = | link_hino = | aniversário = 21 de março | fundação = | emancipação = | gentílico = lindoiano | lema = Aqua pura vita longa "Água pura, vida longa" | padroeiro = Nossa Senhora das Brotas | prefeito = Luciano Francisco de Godoi Lopes | partido = PSDB | fim_mandato = 2024 | mapa = SaoPaulo Municip Lindoia.svg | latP = S | latG = 22 | latM = 31 | latS = 22 | lonP = O | lonG = 46 | lonM = 39 | lonS = 00 | estado = São Paulo | região_intermediária = Campinas | data_região_intermediária = IBGE/2017 | região_imediata = 